In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sn
import matplotlib.pyplot as plt
from scipy import stats 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import winsound
from sklearn import tree
from keras import regularizers
import time
import sklearn.metrics
import seaborn as sb
from mlxtend.plotting import plot_confusion_matrix

## Data preparation

In [2]:
data = pd.read_csv('extracted_features.csv')
data = data.drop(['tourney_id'
                 ,'winner_id'
                 ,'loser_id',
                 'draw_size','proba_elo',
                 'w_ace','w_df','w_svpt','w_1stIn','w_1stWon','w_SvGms','w_bpSaved','w_bpFaced','w_2ndWon','l_2ndWon',
                 'l_ace','l_df','l_svpt','l_1stIn','l_1stWon','l_SvGms','l_bpSaved','l_bpFaced','index'],axis=1)

In [3]:
data

,tourney_name,surface,tourney_level,tourney_date,winner_name,winner_hand,winner_ht,winner_age,loser_name,loser_hand,loser_ht,loser_age,score,best_of,round,winner_rank,loser_rank,winner_tournaments_won,loser_tournaments_won,winner_aces,winner_double_faults,winner_first_serve_in,winner_first_serve_won,winner_second_serve_won,loser_aces,loser_double_faults,loser_first_serve_in,loser_first_serve_won,loser_second_serve_won,winner_games_won,loser_games_won,winner_games_played,loser_games_played,winner_games_round_won,loser_games_round_won,winner_games_tourney_won,loser_games_tourney_won,winner_games_surface_won,loser_games_surface_won,winner_games_opponent_won,loser_games_opponent_won,winner_games_hand_won,loser_games_hand_won,winner_break_points_saved,loser_break_points_saved,winner_break_points_converted,loser_break_points_converted,winner_first_serve_return_won,winner_second_serve_return_won,loser_first_serve_return_won,loser_second_serve_return_won,year,winner_elo,loser_elo,winner_glicko,loser_glicko,winner_RD,loser_RD,winner_matches_won,loser_matches_won,winner_matches_lost,loser_matches_lost,winner_consecutive_matches_won,loser_consecutive_matches_won,winner_consecutive_matches_lost,loser_consecutive_matches_lost,winner_weeks_inactive,loser_weeks_inactive,winner_games_played_last_weeks,loser_games_played_last_weeks,winner_tiebreaks_won_year,loser_tiebreaks_won_year,winner_home,loser_home
0,Auckland,Hard,Other,19920106,Jacco Eltingh,R,188.0,21.3,Karel Novacek,R,190.000000,26.7,7-6 7-6,3,R32,110.0,8.0,0,7,0.086037,0.042666,0.608604,0.692352,0.475676,0.069455,0.033623,0.564325,0.727273,0.515942,0.445652,0.531766,920.0,1999.0,0.445455,0.515780,0.000000,0.529412,0.422604,0.482105,0.000000,0.000000,0.476813,0.516833,0.566308,0.630975,0.333333,0.426644,0.265222,0.467049,0.310444,0.512026,1992,1492.143440,1883.772450,1703.326523,2057.531359,80.648832,60.056906,14,158,28,128,0,0,12,4,1,7,20.0,0.0,0.583333,0.571429,0,0
1,Sydney Outdoor,Hard,Other,19920106,Todd Woodbridge,R,178.0,20.7,Michael Stich,R,193.000000,23.2,4-6 6-1 6-1,3,R32,80.0,4.0,0,5,0.025649,0.046771,0.602293,0.668337,0.496206,0.083055,0.039300,0.583930,0.758311,0.557553,0.498185,0.557871,1102.0,2255.0,0.483019,0.535828,0.367347,0.555556,0.503173,0.550479,0.333333,0.666667,0.490826,0.543283,0.563467,0.689487,0.413249,0.412892,0.297468,0.530738,0.316658,0.470344,1992,1750.186463,2054.554908,1912.885802,2172.543768,67.346775,59.535010,45,107,54,58,0,0,1,1,3,3,0.0,0.0,0.500000,0.613636,1,0
2,Auckland,Hard,Other,19920106,Grant Connell,L,185.0,26.1,Lars Jonsson,R,188.000000,21.5,6-3 6-4,3,R32,78.0,101.0,0,0,0.063127,0.035693,0.582596,0.710380,0.506714,0.031285,0.020112,0.707635,0.623684,0.482803,0.490269,0.497531,1079.0,810.0,0.501502,0.504212,0.520231,0.542373,0.492644,0.513344,0.000000,0.000000,0.502387,0.465318,0.663366,0.575862,0.351648,0.453875,0.305192,0.464399,0.328553,0.522593,1992,1722.873014,1716.350299,1886.557197,1871.293999,73.317826,71.753235,56,37,73,44,0,0,1,3,9,1,0.0,16.0,0.454545,0.125000,0,0
3,Sydney Outdoor,Hard,Other,19920106,Thomas Muster,L,180.0,24.2,Grant Doyle,R,180.000000,17.9,6-4 6-2,3,R32,37.0,627.0,10,0,0.012874,0.019871,0.735236,0.622764,0.525370,0.038961,0.045455,0.571429,0.647727,0.439394,0.530864,0.404255,1134.0,47.0,0.535517,0.500000,0.000000,0.000000,0.510758,0.404255,0.000000,0.000000,0.541111,0.200000,0.600543,0.619048,0.477387,0.444444,0.371954,0.540950,0.263736,0.395833,1992,1869.826274,1461.912360,2045.265919,1414.532220,65.193945,275.641148,187,0,103,2,0,0,2,2,1,1,32.0,32.0,0.500000,0.000000,0,1
4,Auckland,Hard,Other,19920106,Christian Miniussi,R,185.0,24.5,James Greenhalgh,R,183.000000,16.8,6-4 6-2,3,R32,82.0,1059.0,1,0,0.022564,0.035897,0.631795,0.665584,0.470752,0.018868,0.094340,0.377358,0.550000,0.424242,0.529716,0.294118,387.0,17.0,0.471885,0.294118,0.000000,0.000000,0.521127,0.294118,0.000000,0.000000,0.479270,0.294118,0.489583,0.375000,0.435115,0.166667,0.359806,0.559783,0.230769,0.423077,1992,1823.527461,1465.858446,1938.920422,1

In [4]:
data['target'] = 1

In [5]:
s = data.sample(44681)
s['target'] = 0

In [139]:
def swap_columns(df, col1, col2):
    c1 = df[col1]
    c2 = df[col2]
    df[col1]=c2
    df[col2]=c1
    return df

In [7]:
s=swap_columns(s, 'winner_ht', 'loser_ht')
s=swap_columns(s, 'winner_age', 'loser_age')
s=swap_columns(s, 'winner_hand', 'loser_hand')

s=swap_columns(s, 'winner_rank', 'loser_rank')
s=swap_columns(s, 'winner_elo', 'loser_elo')
s=swap_columns(s, 'winner_glicko', 'loser_glicko')
s=swap_columns(s, 'winner_RD', 'loser_RD')

s=swap_columns(s, 'winner_tournaments_won', 'loser_tournaments_won')
s=swap_columns(s, 'winner_games_played', 'loser_games_played')
s=swap_columns(s, 'winner_games_won', 'loser_games_won')
s=swap_columns(s, 'winner_games_round_won', 'loser_games_round_won')
s=swap_columns(s, 'winner_games_tourney_won', 'loser_games_tourney_won')
s=swap_columns(s, 'winner_games_surface_won', 'loser_games_surface_won')
s=swap_columns(s, 'winner_games_opponent_won', 'loser_games_opponent_won')
s=swap_columns(s, 'winner_games_hand_won', 'loser_games_hand_won')
s=swap_columns(s, 'winner_matches_won', 'loser_matches_won')
s=swap_columns(s, 'winner_matches_lost', 'loser_matches_lost')

s=swap_columns(s, 'winner_consecutive_matches_won', 'loser_consecutive_matches_won')
s=swap_columns(s, 'winner_consecutive_matches_lost', 'loser_consecutive_matches_lost')
s=swap_columns(s, 'winner_weeks_inactive', 'loser_weeks_inactive')
s=swap_columns(s, 'winner_games_played_last_weeks', 'loser_games_played_last_weeks')

s=swap_columns(s, 'winner_first_serve_in', 'loser_first_serve_in')
s=swap_columns(s, 'winner_first_serve_won', 'loser_first_serve_won')
s=swap_columns(s, 'winner_second_serve_won', 'loser_second_serve_won')
s=swap_columns(s, 'winner_aces', 'loser_aces')
s=swap_columns(s, 'winner_double_faults', 'loser_double_faults')

s=swap_columns(s, 'winner_break_points_converted', 'loser_break_points_converted')
s=swap_columns(s, 'winner_first_serve_return_won', 'loser_first_serve_return_won')
s=swap_columns(s, 'winner_second_serve_return_won', 'loser_second_serve_return_won')

s=swap_columns(s, 'winner_break_points_saved', 'loser_break_points_saved')
s=swap_columns(s, 'winner_tiebreaks_won_year', 'loser_tiebreaks_won_year')
s=swap_columns(s, 'winner_home', 'loser_home')

In [8]:
data.update(s)

In [9]:
data

,tourney_name,surface,tourney_level,tourney_date,winner_name,winner_hand,winner_ht,winner_age,loser_name,loser_hand,loser_ht,loser_age,score,best_of,round,winner_rank,loser_rank,winner_tournaments_won,loser_tournaments_won,winner_aces,winner_double_faults,winner_first_serve_in,winner_first_serve_won,winner_second_serve_won,loser_aces,loser_double_faults,loser_first_serve_in,loser_first_serve_won,loser_second_serve_won,winner_games_won,loser_games_won,winner_games_played,loser_games_played,winner_games_round_won,loser_games_round_won,winner_games_tourney_won,loser_games_tourney_won,winner_games_surface_won,loser_games_surface_won,winner_games_opponent_won,loser_games_opponent_won,winner_games_hand_won,loser_games_hand_won,winner_break_points_saved,loser_break_points_saved,winner_break_points_converted,loser_break_points_converted,winner_first_serve_return_won,winner_second_serve_return_won,loser_first_serve_return_won,loser_second_serve_return_won,year,winner_elo,loser_elo,winner_glicko,loser_glicko,winner_RD,loser_RD,winner_matches_won,loser_matches_won,winner_matches_lost,loser_matches_lost,winner_consecutive_matches_won,loser_consecutive_matches_won,winner_consecutive_matches_lost,loser_consecutive_matches_lost,winner_weeks_inactive,loser_weeks_inactive,winner_games_played_last_weeks,loser_games_played_last_weeks,winner_tiebreaks_won_year,loser_tiebreaks_won_year,winner_home,loser_home,target
0,Auckland,Hard,Other,19920106.0,Jacco Eltingh,R,188.000000,21.3,Karel Novacek,R,190.000000,26.7,7-6 7-6,3.0,R32,110.0,8.0,0.0,7.0,0.086037,0.042666,0.608604,0.692352,0.475676,0.069455,0.033623,0.564325,0.727273,0.515942,0.445652,0.531766,920.0,1999.0,0.445455,0.515780,0.000000,0.529412,0.422604,0.482105,0.000000,0.000000,0.476813,0.516833,0.566308,0.630975,0.333333,0.426644,0.265222,0.467049,0.310444,0.512026,1992.0,1492.143440,1883.772450,1703.326523,2057.531359,80.648832,60.056906,14.0,158.0,28.0,128.0,0.0,0.0,12.0,4.0,1.0,7.0,20.0,0.0,0.583333,0.571429,0.0,0.0,1.0
1,Sydney Outdoor,Hard,Other,19920106.0,Todd Woodbridge,R,178.000000,20.7,Michael Stich,R,193.000000,23.2,4-6 6-1 6-1,3.0,R32,80.0,4.0,0.0,5.0,0.025649,0.046771,0.602293,0.668337,0.496206,0.083055,0.039300,0.583930,0.758311,0.557553,0.498185,0.557871,1102.0,2255.0,0.483019,0.535828,0.367347,0.555556,0.503173,0.550479,0.333333,0.666667,0.490826,0.543283,0.563467,0.689487,0.413249,0.412892,0.297468,0.530738,0.316658,0.470344,1992.0,1750.186463,2054.554908,1912.885802,2172.543768,67.346775,59.535010,45.0,107.0,54.0,58.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,0.0,0.500000,0.613636,1.0,0.0,1.0
2,Auckland,Hard,Other,19920106.0,Grant Connell,L,185.000000,26.1,Lars Jonsson,R,188.000000,21.5,6-3 6-4,3.0,R32,78.0,101.0,0.0,0.0,0.063127,0.035693,0.582596,0.710380,0.506714,0.031285,0.020112,0.707635,0.623684,0.482803,0.490269,0.497531,1079.0,810.0,0.501502,0.504212,0.520231,0.542373,0.492644,0.513344,0.000000,0.000000,0.502387,0.465318,0.663366,0.575862,0.351648,0.453875,0.305192,0.464399,0.328553,0.522593,1992.0,1722.873014,1716.350299,1886.557197,1871.293999,73.317826,71.753235,56.0,37.0,73.0,44.0,0.0,0.0,1.0,3.0,9.0,1.0,0.0,16.0,0.454545,0.125000,0.0,0.0,1.0
3,Sydney Outdoor,Hard,Other,19920106.0,Thomas Muster,R,180.000000,17.9,Grant Doyle,L,180.000000,24.2,6-4 6-2,3.0,R32,627.0,37.0,0.0,10.0,0.038961,0.045455,0.571429,0.647727,0.439394,0.012874,0.019871,0.735236,0.622764,0.525370,0.404255,0.530864,47.0,1134.0,0.500000,0.535517,0.000000,0.000000,0.404255,0.510758,0.000000,0.000000,0.200000,0.541111,0.619048,0.600543,0.444444,0.477387,0.263736,0.395833,0.371954,0.540950,1992.0,1461.912360,1869.826274,1414.532220,2045.265919,275.641148,65.193945,0.0,187.0,2.0,103.0,0.0,0.0,2.0,2.0,1.0,1.0,32.0,32.0,0.000000,0.500000,1.0,0.0,0.0
4,Auckland,Hard,Other,19920106.0,Christian Miniussi,R,185.000000,24.5,James Greenhalgh,R,183.000000,16.8,6-4 6-2,3.0,R32,82.0,1059.0,1.0,0.0,0.022564,0.035897,0.631795,0.665584,0.470752,0.018868,0.094340,0.377358,0.550000,0.424242,0.529716,0.294118,387.0,17.0,0.471885,0.2

In [10]:
data.to_csv('data_with_names.csv', index=False)

In [11]:
data_symmetric = pd.DataFrame()

In [12]:
data_symmetric['surface'] = data['surface']
data_symmetric['tourney_level'] = data['tourney_level']
data_symmetric['round'] = data['round']
data_symmetric['best_of'] = data['best_of']
data_symmetric['year'] = data['year']

data_symmetric['height'] = data['winner_ht']-data['loser_ht']
data_symmetric['age'] = data['winner_age']-data['loser_age']
data_symmetric['player1_hand'] = data['winner_hand']
data_symmetric['player2_hand'] = data['loser_hand']

data_symmetric['rank'] = data['winner_rank']-data['loser_rank']
data_symmetric['elo'] = data['winner_elo']-data['loser_elo']
data_symmetric['glicko'] = data['winner_glicko']-data['loser_glicko']
data_symmetric['RD'] = data['winner_RD']-data['loser_RD']
data_symmetric['tournaments_won'] = data['winner_tournaments_won']-data['loser_tournaments_won']
data_symmetric['games_played'] = data['winner_games_played']-data['loser_games_played']
data_symmetric['games_won'] = data['winner_games_won']-data['loser_games_won']
data_symmetric['games_round_won'] = data['winner_games_round_won']-data['loser_games_round_won']
data_symmetric['games_tourney_won'] = data['winner_games_tourney_won']-data['loser_games_tourney_won']
data_symmetric['games_surface_won'] = data['winner_games_surface_won']-data['loser_games_surface_won']
data_symmetric['games_opponent_won'] = data['winner_games_opponent_won']-data['loser_games_opponent_won']
data_symmetric['games_hand_won'] = data['winner_games_hand_won']-data['loser_games_hand_won']
data_symmetric['matches_won'] = data['winner_matches_won']-data['loser_matches_won']
data_symmetric['matches_lost'] = data['winner_matches_lost']-data['loser_matches_lost']

data_symmetric['consecutive_matches_won'] = data['winner_consecutive_matches_won']-data['loser_consecutive_matches_won']
data_symmetric['consecutive_matches_lost'] = data['winner_consecutive_matches_lost']-data['loser_consecutive_matches_lost']
data_symmetric['weeks_inactive'] = data['winner_weeks_inactive']-data['loser_weeks_inactive']
data_symmetric['games_played_last_weeks'] = data['winner_games_played_last_weeks']-data['loser_games_played_last_weeks']

data_symmetric['first_serve_in'] = data['winner_first_serve_in']-data['loser_first_serve_in']
data_symmetric['first_serve_won'] = data['winner_first_serve_won']-data['loser_first_serve_won']
data_symmetric['second_serve_won'] = data['winner_second_serve_won']-data['loser_second_serve_won']
data_symmetric['aces'] = data['winner_aces']-data['loser_aces']
data_symmetric['double_faults'] = data['winner_double_faults']-data['loser_double_faults']

data_symmetric['break_points_converted'] = data['winner_break_points_converted']-data['loser_break_points_converted']
data_symmetric['first_serve_return_won'] = data['winner_first_serve_return_won']-data['loser_first_serve_return_won']
data_symmetric['second_serve_return_won'] = data['winner_second_serve_return_won']-data['loser_second_serve_return_won']

data_symmetric['break_points_saved'] = data['winner_break_points_saved']-data['loser_break_points_saved']
data_symmetric['tiebreaks_won'] = data['winner_tiebreaks_won_year']-data['loser_tiebreaks_won_year']
data_symmetric['player1_home'] = data['winner_home']
data_symmetric['player2_home'] = data['loser_home']

data_symmetric['target'] = data['target']

In [13]:
data_symmetric

,surface,tourney_level,round,best_of,year,height,age,player1_hand,player2_hand,rank,elo,glicko,RD,tournaments_won,games_played,games_won,games_round_won,games_tourney_won,games_surface_won,games_opponent_won,games_hand_won,matches_won,matches_lost,consecutive_matches_won,consecutive_matches_lost,weeks_inactive,games_played_last_weeks,first_serve_in,first_serve_won,second_serve_won,aces,double_faults,break_points_converted,first_serve_return_won,second_serve_return_won,break_points_saved,tiebreaks_won,player1_home,player2_home,target
0,Hard,Other,R32,3.0,1992.0,-2.000000,-5.4,R,R,102.0,-391.629010,-354.204836,20.591926,-7.0,-1079.0,-0.086114,-0.070326,-0.529412,-0.059501,0.000000,-0.040020,-144.0,-100.0,0.0,8.0,-6.0,20.0,0.044278,-0.034920,-0.040266,0.016581,0.009043,-0.093311,-0.045221,-0.044977,-0.064667,0.011905,0.0,0.0,1.0
1,Hard,Other,R32,3.0,1992.0,-15.000000,-2.5,R,R,76.0,-304.368445,-259.657966,7.811765,-5.0,-1153.0,-0.059686,-0.052809,-0.188209,-0.047307,-0.333333,-0.052457,-62.0,-4.0,0.0,0.0,0.0,0.0,0.018363,-0.089974,-0.061346,-0.057406,0.007471,0.000357,-0.019189,0.060394,-0.126019,-0.113636,1.0,0.0,1.0
2,Hard,Other,R32,3.0,1992.0,-3.000000,4.6,L,R,-23.0,6.522715,15.263198,1.564591,0.0,269.0,-0.007262,-0.002710,-0.022142,-0.020700,0.000000,0.037070,19.0,29.0,0.0,-2.0,8.0,-16.0,-0.125039,0.086696,0.023911,0.031842,0.015581,-0.102226,-0.023361,-0.058195,0.087504,0.329545,0.0,0.0,1.0
3,Hard,Other,R32,3.0,1992.0,0.000000,-6.3,R,L,590.0,-407.913914,-630.733700,210.447203,-10.0,-1087.0,-0.126609,-0.035517,0.000000,-0.106503,0.000000,-0.341111,-187.0,-101.0,0.0,0.0,0.0,0.0,-0.163808,0.024964,-0.085976,0.026087,0.025583,-0.032942,-0.108218,-0.145116,0.018504,-0.500000,1.0,0.0,0.0
4,Hard,Other,R32,3.0,1992.0,2.000000,7.7,R,R,-977.0,357.669015,523.845029,-182.323633,1.0,370.0,0.235598,0.177768,0.000000,0.227009,0.000000,0.185153,40.0,50.0,4.0,-1.0,8.0,-17.0,0.254436,0.115584,0.046510,0.003696,-0.058442,0.268448,0.129036,0.136706,0.114583,0.000000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89358,Hard,Season-ending events,RR,5.0,2023.0,0.000000,-0.9,R,R,-79.0,260.293008,420.628507,-107.096833,1.0,635.0,0.097883,-0.416667,-0.416667,0.061555,0.000000,0.079761,15.0,5.0,0.0,2.0,4.0,-48.0,-0.066103,0.093411,0.008719,0.044800,0.020008,0.136290,0.028814,-0.025037,0.200549,0.071429,0.0,0.0,1.0
89359,Hard,Season-ending events,RR,5.0,2023.0,0.694044,-2.1,R,R,-64.0,168.105313,195.917047,-46.780356,1.0,582.0,0.005795,-0.053571,-0.053571,0.029331,0.000000,0.040626,13.0,6.0,0.0,0.0,0.0,-26.0,0.035927,0.039339,-0.012807,0.028885,0.010465,-0.023421,-0.050615,0.011312,-0.041368,0.433333,0.0,0.0,1.0
89360,Hard,Season-ending events,RR,5.0,2023.0,5.000000,-1.8,R,L,-58.0,24.557854,28.750428,-9.400948,1.0,378.0,0.024076,0.016946,0.045056,0.004641,-0.076923,-0.028848,1.0,1.0,2.0,-1.0,0.0,7.0,0.054761,-0.025247,-0.021718,-0.008944,0.008539,0.076993,0.029429,-0.030891,0.052181,-0.015385,0.0,0.0,1.0
89361,Hard,Season-ending events,SF,5.0,2023.0,-0.694044,0.1,R,R,34.0,-198.764456,-197.976984,15.325615,-1.0,-83.0,-0.050281,-0.445378,-0.044478,-0.057989,0.000000,-0.050511,-10.0,3.0,-2.0,0.0,0.0,8.0,-0.013541,-0.064984,-0.005405,-0.042905,-0.009084,-0.035650,0.015500,0.002548,-0.012705,0.036765,0.0,0.0,0.0


In [14]:
for col in ['surface', 'tourney_level', 'round', 'player1_hand','player2_hand']:
    data_symmetric[col] = data_symmetric[col].astype('category')
    
for col in ['player1_home', 'player2_home', 'best_of', 'year','rank','tournaments_won','games_played','matches_won','target',
           'matches_lost','consecutive_matches_won','consecutive_matches_lost','weeks_inactive','games_played_last_weeks']:
    data_symmetric[col] = data_symmetric[col].astype('int64')

In [15]:
data_symmetric['Y'] = data_symmetric['year']

In [16]:
num_cols = ['best_of', 'year', 'height', 'age',
       'rank', 'elo', 'glicko', 'RD', 'tournaments_won', 'games_played',
       'games_won', 'games_round_won', 'games_tourney_won',
       'games_surface_won', 'games_opponent_won', 'games_hand_won',
       'matches_won', 'matches_lost', 'consecutive_matches_won',
       'consecutive_matches_lost', 'weeks_inactive', 'games_played_last_weeks',
       'first_serve_in', 'first_serve_won',
       'second_serve_won', 'aces', 'double_faults', 'break_points_converted',
       'first_serve_return_won', 'second_serve_return_won',
       'break_points_saved', 'tiebreaks_won']

In [17]:
cat_cols = ['surface','tourney_level','round','player1_hand','player2_hand']

In [18]:
data_symmetric_enc = data_symmetric.copy()

In [19]:
for col in num_cols:
    data_symmetric_enc[col] = stats.zscore(data_symmetric_enc[col])

In [20]:
data_symmetric_enc = pd.get_dummies(data_symmetric_enc)

In [21]:
data_symmetric.to_csv('data_symmetric.csv', index=False)
data_symmetric_enc.to_csv('data_symmetric_enc.csv', index=False)

## Models

In [22]:
def check_performance_window_proba(clf, df):
    start = time.time()
    test_scores = []
    train_scores = []
    test_probas = []
    for year in range(2006,2024):
        train = df[(df['Y']<year) & (df['Y']>=year-7)]
        test = df[df['Y']==year]
        train = train.drop(['Y'], axis=1)
        test = test.drop(['Y'], axis=1)

        X_train = train.drop(['target'], axis=1)
        y_train = train['target']

        X_test = test.drop(['target'], axis=1)
        y_test = test['target']

        #tutaj feature selection

        clf.fit(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        train_score = clf.score(X_train, y_train)
        test_scores.append(test_score)
        train_scores.append(train_score)
        
        test_proba = clf.predict_proba(X_test)
        test_probas.extend(test_proba)
    
    return test_scores, train_scores, np.array(test_probas), time.time()-start

### LR

In [23]:
columns_lr2 = ['year',
  'height',
  'age',
  'rank',
  'elo',
  'glicko',
  'games_played',
  'games_won',
  'games_round_won',
  'games_tourney_won',
  'games_surface_won',
  'games_opponent_won',
  'games_hand_won',
  'matches_won',
  'matches_lost',
  'consecutive_matches_won',
  'consecutive_matches_lost',
  'weeks_inactive',
  'games_played_last_weeks',
  'first_serve_in',
  'first_serve_won',
  'second_serve_won',
  'aces',
  'double_faults',
  'break_points_converted',
  'first_serve_return_won',
  'break_points_saved',
  'tiebreaks_won',
  'player1_home',
  'player2_home',
  'surface_Carpet',
  'surface_Clay',
  'surface_Grass',
  'surface_Hard',
  'tourney_level_Grand Slam',
  'tourney_level_Masters 1000',
  'tourney_level_Other',
  'tourney_level_Season-ending events',
  'round_F',
  'round_QF',
  'round_R128',
  'round_R16',
  'round_R32',
  'round_R64',
  'round_RR',
  'round_SF',
  'player1_hand_R',
  'player2_hand_L',
  'player2_hand_R']

In [24]:
lr = LogisticRegression(max_iter=600,penalty='none')
columns_lr2.extend(['Y','target'])
test_lr, train_lr, test_lr_proba, time_lr = check_performance_window_proba(lr, data_symmetric_enc[columns_lr2])
print(np.mean(test_lr), np.mean(train_lr))

0.6818637871343887 0.6857434901838443


### knn

In [25]:
columns_knn2 = ['best_of',
  'year',
  'height',
  'age',
  'rank',
  'elo',
  'glicko',
  'tournaments_won',
  'games_played',
  'games_won',
  'games_round_won',
  'games_tourney_won',
  'games_surface_won',
  'games_opponent_won',
  'games_hand_won',
  'matches_won',
  'matches_lost',
  'consecutive_matches_won',
  'consecutive_matches_lost',
  'games_played_last_weeks',
  'first_serve_in',
  'first_serve_won',
  'second_serve_won',
  'aces',
  'double_faults',
  'break_points_converted',
  'first_serve_return_won',
  'second_serve_return_won',
  'break_points_saved',
  'tiebreaks_won',
  'player2_home',
  'surface_Carpet',
  'surface_Clay',
  'surface_Grass',
  'surface_Hard',
  'tourney_level_Masters 1000',
  'tourney_level_Other',
  'tourney_level_Season-ending events',
  'round_F',
  'round_QF',
  'round_R16',
  'round_R32',
  'round_SF',
  'player1_hand_L',
  'player1_hand_R',
  'player2_hand_R']

In [26]:
columns_knn2.extend(['Y','target'])
knn = KNeighborsClassifier(n_neighbors=126)
test_knn, train_knn, test_knn_proba, time_knn = check_performance_window_proba(knn, data_symmetric_enc[columns_knn2])
print(np.mean(test_knn), np.mean(train_knn))

0.6792089788326937 0.6866953575043033


### svm

In [27]:
columns_svm2 = ['best_of',
  'year',
  'height',
  'age',
  'rank',
  'elo',
  'glicko',
  'RD',
  'tournaments_won',
  'games_played',
  'games_won',
  'games_round_won',
  'games_tourney_won',
  'games_surface_won',
  'games_opponent_won',
  'games_hand_won',
  'matches_won',
  'matches_lost',
  'consecutive_matches_lost',
  'weeks_inactive',
  'games_played_last_weeks',
  'first_serve_in',
  'first_serve_won',
  'second_serve_won',
  'aces',
  'double_faults',
  'break_points_converted',
  'first_serve_return_won',
  'second_serve_return_won',
  'break_points_saved',
  'tiebreaks_won',
  'player1_home',
  'player2_home',
  'surface_Carpet',
  'surface_Clay',
  'surface_Grass',
  'surface_Hard',
  'tourney_level_Grand Slam',
  'tourney_level_Masters 1000',
  'tourney_level_Other',
  'tourney_level_Season-ending events',
  'round_F',
  'round_QF',
  'round_R128',
  'round_R16',
  'round_R32',
  'round_R64',
  'round_RR',
  'round_SF',
  'player1_hand_R',
  'player2_hand_R']

In [28]:
columns_svm2.extend(['Y','target'])
sv = svm.SVC(C=1, kernel='rbf', gamma='scale',probability=True)
test_svm, train_svm, test_svm_proba, time_svm = check_performance_window_proba(sv, data_symmetric_enc[columns_svm2])
print(np.mean(test_svm), np.mean(train_svm))

0.6832075659382487 0.7175648344144147


### nn

In [29]:
columns_all = data_symmetric_enc.columns.values.tolist()
columns_all.remove('target')
columns_all.remove('Y')

In [30]:
    #all features  
    test_scores = []
    train_scores = []
    test_nn_proba = []
    df = data_symmetric_enc
    start = time.time()
    for year in range(2006,2024):
        train = df[(df['Y']<year) & (df['Y']>=year-7)]
        test = df[df['Y']==year]
        train = train.drop(['Y'], axis=1)
        test = test.drop(['Y'], axis=1)

        X_train = train.drop(['target'], axis=1)
        y_train = train['target']

        X_test = test.drop(['target'], axis=1)
        y_test = test['target']

        X_test = X_test[columns_all]
        X_train = X_train[columns_all]
        
        ann = Sequential()
        ann.add(Dense(512, activation="tanh",kernel_regularizer=regularizers.L1(0.00001)))
        ann.add(Dropout(0.4))
        ann.add(Dense(1, activation="sigmoid"))

        ann.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        ann.fit(X_train, y_train, epochs=40, batch_size=32)
        
        test_score = ann.evaluate(X_test,  y_test)[1]
        test_scores.append(test_score)
        
        train_score = ann.evaluate(X_train,  y_train)[1]
        train_scores.append(train_score)
        
        test_nn_proba.extend(ann.predict(X_test))
    
    time_nn = time.time() - start
    test_nn = test_scores
    train_nn = train_scores
    np.mean(test_scores), np.mean(train_scores)



Epoch 1/40


628/628 [==============================] - 2s 1ms/step - loss: 0.6468 - accuracy: 0.6464
Epoch 2/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6355 - accuracy: 0.6573
Epoch 3/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6300 - accuracy: 0.6575
Epoch 4/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6271 - accuracy: 0.6597
Epoch 5/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6258 - accuracy: 0.6591
Epoch 6/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6239 - accuracy: 0.6593
Epoch 7/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6229 - accuracy: 0.6628
Epoch 8/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6209 - accuracy: 0.6630
Epoch 9/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6210 - accuracy: 0.6595
Epoch 10/40
628/628 [==============================] - 1s 1ms/step - loss: 0.6197 - accuracy: 0.

608/608 [==============================] - 1s 1ms/step - loss: 0.5996 - accuracy: 0.6816
Epoch 27/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5984 - accuracy: 0.6804
Epoch 28/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5977 - accuracy: 0.6841
Epoch 29/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5977 - accuracy: 0.6826
Epoch 30/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5985 - accuracy: 0.6798
Epoch 31/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5983 - accuracy: 0.6790
Epoch 32/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5997 - accuracy: 0.6813
Epoch 33/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5987 - accuracy: 0.6821
Epoch 34/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5992 - accuracy: 0.6818
Epoch 35/40
608/608 [==============================] - 1s 1ms/step - loss: 0.5983 - accuracy: 0.6800
Ep

588/588 [==============================] - 1s 1ms/step - loss: 0.5830 - accuracy: 0.6973
Epoch 19/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5836 - accuracy: 0.6967
Epoch 20/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5822 - accuracy: 0.6967
Epoch 21/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5829 - accuracy: 0.6961
Epoch 22/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5828 - accuracy: 0.6966
Epoch 23/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5814 - accuracy: 0.6951
Epoch 24/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5823 - accuracy: 0.6958
Epoch 25/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5821 - accuracy: 0.6975
Epoch 26/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5815 - accuracy: 0.6975
Epoch 27/40
588/588 [==============================] - 1s 1ms/step - loss: 0.5825 - accuracy: 0.6972
Ep

562/562 [==============================] - 1s 1ms/step - loss: 0.5826 - accuracy: 0.6967
Epoch 11/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5811 - accuracy: 0.6975
Epoch 12/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5802 - accuracy: 0.6996
Epoch 13/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5800 - accuracy: 0.6982
Epoch 14/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5795 - accuracy: 0.6979
Epoch 15/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5790 - accuracy: 0.7005
Epoch 16/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5793 - accuracy: 0.7011
Epoch 17/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5782 - accuracy: 0.7016
Epoch 18/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5805 - accuracy: 0.6960
Epoch 19/40
562/562 [==============================] - 1s 1ms/step - loss: 0.5780 - accuracy: 0.7012
Ep

518/518 [==============================] - 1s 1ms/step - loss: 0.6154 - accuracy: 0.6706
Epoch 3/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6125 - accuracy: 0.6727
Epoch 4/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6093 - accuracy: 0.6774
Epoch 5/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6075 - accuracy: 0.6744
Epoch 6/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6060 - accuracy: 0.6775
Epoch 7/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6040 - accuracy: 0.6793
Epoch 8/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6043 - accuracy: 0.6755
Epoch 9/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6026 - accuracy: 0.6790
Epoch 10/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6013 - accuracy: 0.6784
Epoch 11/40
518/518 [==============================] - 1s 1ms/step - loss: 0.6018 - accuracy: 0.6818
Epoch 12/

(0.6834191249476539, 0.6908976137638092)

### nb

In [31]:
columns_nb2 = ['year',
  'height',
  'age',
  'rank',
  'elo',
  'glicko',
  'RD',
  'tournaments_won',
  'games_won',
  'games_round_won',
  'games_tourney_won',
  'games_surface_won',
  'games_opponent_won',
  'games_hand_won',
  'consecutive_matches_won',
  'weeks_inactive',
  'games_played_last_weeks',
  'first_serve_in',
  'second_serve_won',
  'aces',
  'double_faults',
  'break_points_converted',
  'first_serve_return_won',
  'second_serve_return_won',
  'break_points_saved',
  'tiebreaks_won',
  'player1_home',
  'surface_Carpet',
  'surface_Clay',
  'surface_Hard',
  'tourney_level_Masters 1000',
  'tourney_level_Other',
  'round_QF',
  'round_R128',
  'round_R64',
  'player1_hand_L',
  'player1_hand_R',
  'player2_hand_L',
  'player2_hand_R']

In [32]:
columns_nb2.extend(['Y','target'])
gnb = GaussianNB()
test_nb, train_nb, test_nb_proba, time_nb = check_performance_window_proba(gnb, data_symmetric_enc[columns_nb2])
print(np.mean(test_nb), np.mean(train_nb))

0.6755859241253683 0.6788252610402311


### xgb

In [33]:
xg = xgb.XGBClassifier(enable_categorical=True,learning_rate=0.1,n_estimators=100,subsample=1,max_depth=3,min_samples_split=2)
test_xgb, train_xgb, test_xgb_proba, time_xgb = check_performance_window_proba(xg, data_symmetric)
print(np.mean(test_xgb), np.mean(train_xgb))

C:\Users\krych\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:47:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\krych\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:47:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\krych\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:160: UserWarning: [10:47:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "min_samples_split" } are

0.682517519939271 0.7022792619485629


### dt

In [34]:
columns_corr = data_symmetric_enc.columns.values.tolist()
columns_corr.remove('target')
columns_corr.remove('Y')
columns_corr.remove('age')
columns_corr.remove('elo')
columns_corr.remove('tournaments_won')
columns_corr.remove('first_serve_in')
columns_corr.remove('first_serve_won')
columns_corr.remove('second_serve_won')
columns_corr.remove('first_serve_return_won')
columns_corr.remove('second_serve_return_won')
columns_corr.remove('break_points_saved')

In [35]:
columns_corr.extend(['Y','target'])
dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=2, max_features=None) 
test_dt, train_dt, test_dt_proba, time_dt = check_performance_window_proba(dt, data_symmetric_enc[columns_corr])
print(np.mean(test_dt), np.mean(train_dt))

0.6693727434107816 0.6764954008140321


### rf

In [36]:
rf = RandomForestClassifier(criterion='gini',n_estimators=80,max_depth=8,min_samples_split=4,max_features='sqrt')
test_rf, train_rf, test_rf_proba, time_rf = check_performance_window_proba(rf, data_symmetric_enc)
print(np.mean(test_rf), np.mean(train_rf))

0.680616182373373 0.7305436259446936


## All probabilities

In [37]:
lr_predicted_all = test_lr_proba[:,1]
dt_predicted_all = test_dt_proba[:,1]
rf_predicted_all = test_rf_proba[:,1]
xgb_predicted_all = test_xgb_proba[:,1]
knn_predicted_all = test_knn_proba[:,1]
svm_predicted_all = test_svm_proba[:,1]
nb_predicted_all = test_nb_proba[:,1]
nn_predicted_all = np.array(test_nn_proba)

In [38]:
targets = data_symmetric_enc[data_symmetric_enc['Y']>2005]['target']

In [39]:
probabilities_all_models = pd.DataFrame()
probabilities_all_models['lr'] = lr_predicted_all
probabilities_all_models['dt'] = dt_predicted_all
probabilities_all_models['rf'] = rf_predicted_all
probabilities_all_models['xgb'] = xgb_predicted_all
probabilities_all_models['knn'] = knn_predicted_all
probabilities_all_models['svm'] = svm_predicted_all
probabilities_all_models['nb'] = nb_predicted_all
probabilities_all_models['nn'] = nn_predicted_all
probabilities_all_models['target'] = np.array(targets)

In [40]:
probabilities_all_models

,lr,dt,rf,xgb,knn,svm,nb,nn,target
0,0.090854,0.270175,0.173877,0.121423,0.222222,0.258194,0.000354,0.090644,0
1,0.944536,0.780952,0.860848,0.863374,0.793651,0.657106,0.999977,0.934234,1
2,0.754095,0.780952,0.739545,0.725770,0.706349,0.674264,0.985837,0.781877,1
3,0.610163,0.780952,0.560366,0.570551,0.523810,0.486693,0.331592,0.570304,0
4,0.692412,0.780952,0.583110,0.672175,0.579365,0.679420,0.927046,0.719571,1
...,...,...,...,...,...,...,...,...,...
45697,0.527965,0.763158,0.766846,0.760311,0.722222,0.648430,0.981146,0.616344,1
45698,0.577001,0.700192,0.671077,0.661541,0.642857,0.600003,0.918794,0.669750,1
45699,0.520632,0.502863,0.523205,0.588997,0.539683,0.532205,0.866218,0.565114,1
45700,0.248567,0.197618,0.274046,0.267798,0.301587,0.232332,0.029877,0.267633,0


In [77]:
data2 = data[data['year']>2005][['tourney_name','surface','tourney_level','tourney_date','winner_name','loser_name','round','score','year','winner_rank','loser_rank']].reset_index()

In [78]:
data2

,index,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,score,year,winner_rank,loser_rank
0,43661,Adelaide,Hard,Other,20060102.0,Lleyton Hewitt,Jan Hernych,R32,4-6 6-2 6-4,2006.0,74.0,4.0
1,43662,Doha,Hard,Other,20060102.0,Roger Federer,Ivo Minar,R32,6-1 6-3,2006.0,1.0,73.0
2,43663,Chennai,Hard,Other,20060102.0,Ivan Ljubicic,Guillermo Garcia Lopez,R32,7-6 7-5,2006.0,9.0,91.0
3,43664,Adelaide,Hard,Other,20060102.0,Philipp Kohlschreiber,Chris Guccione,R32,6-4 6-3,2006.0,156.0,86.0
4,43665,Doha,Hard,Other,20060102.0,Fabrice Santoro,Oliver Marach,R32,6-0 2-6 6-4,2006.0,58.0,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...
45697,89358,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Nardi,RR,2-4 4-3 4-2 1-4 4-2,2023.0,36.0,115.0
45698,89359,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Flavio Cobolli,RR,4-1 4-2 4-2,2023.0,36.0,100.0
45699,89360,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Dominic Stricker,RR,4-2 3-4 4-2 4-3,2023.0,36.0,94.0
45700,89361,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Van Assche,SF,2-4 4-1 4-3 4-3,2023.0,70.0,36.0


In [79]:
concat = pd.concat([data2, probabilities_all_models],axis=1)
concat

,index,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,score,year,winner_rank,loser_rank,lr,dt,rf,xgb,knn,svm,nb,nn,target
0,43661,Adelaide,Hard,Other,20060102.0,Lleyton Hewitt,Jan Hernych,R32,4-6 6-2 6-4,2006.0,74.0,4.0,0.090854,0.270175,0.173877,0.121423,0.222222,0.258194,0.000354,0.090644,0
1,43662,Doha,Hard,Other,20060102.0,Roger Federer,Ivo Minar,R32,6-1 6-3,2006.0,1.0,73.0,0.944536,0.780952,0.860848,0.863374,0.793651,0.657106,0.999977,0.934234,1
2,43663,Chennai,Hard,Other,20060102.0,Ivan Ljubicic,Guillermo Garcia Lopez,R32,7-6 7-5,2006.0,9.0,91.0,0.754095,0.780952,0.739545,0.725770,0.706349,0.674264,0.985837,0.781877,1
3,43664,Adelaide,Hard,Other,20060102.0,Philipp Kohlschreiber,Chris Guccione,R32,6-4 6-3,2006.0,156.0,86.0,0.610163,0.780952,0.560366,0.570551,0.523810,0.486693,0.331592,0.570304,0
4,43665,Doha,Hard,Other,20060102.0,Fabrice Santoro,Oliver Marach,R32,6-0 2-6 6-4,2006.0,58.0,127.0,0.692412,0.780952,0.583110,0.672175,0.579365,0.679420,0.927046,0.719571,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45697,89358,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Nardi,RR,2-4 4-3 4-2 1-4 4-2,2023.0,36.0,115.0,0.527965,0.763158,0.766846,0.760311,0.722222,0.648430,0.981146,0.616344,1
45698,89359,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Flavio Cobolli,RR,4-1 4-2 4-2,2023.0,36.0,100.0,0.577001,0.700192,0.671077,0.661541,0.642857,0.600003,0.918794,0.669750,1
45699,89360,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Dominic Stricker,RR,4-2 3-4 4-2 4-3,2023.0,36.0,94.0,0.520632,0.502863,0.523205,0.588997,0.539683,0.532205,0.866218,0.565114,1
45700,89361,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Van Assche,SF,2-4 4-1 4-3 4-3,2023.0,70.0,36.0,0.248567,0.197618,0.274046,0.267798,0.301587,0.232332,0.029877,0.267633,0


In [80]:
def prob_calc(row,model):
    if row['target']==1:
        return row[model]
    else:
        return 1-row[model]

In [81]:
concat['winner_lr'] = concat.apply(prob_calc,model='lr', axis=1)
concat['loser_lr'] = 1 - concat['winner_lr']

In [82]:
concat['winner_dt'] = concat.apply(prob_calc,model='dt', axis=1)
concat['loser_dt'] = 1 - concat['winner_dt']

In [83]:
concat['winner_rf'] = concat.apply(prob_calc,model='rf', axis=1)
concat['loser_rf'] = 1 - concat['winner_rf']

In [84]:
concat['winner_xgb'] = concat.apply(prob_calc,model='xgb', axis=1)
concat['loser_xgb'] = 1 - concat['winner_xgb']

In [85]:
concat['winner_knn'] = concat.apply(prob_calc,model='knn', axis=1)
concat['loser_knn'] = 1 - concat['winner_knn']

In [86]:
concat['winner_svm'] = concat.apply(prob_calc,model='svm', axis=1)
concat['loser_svm'] = 1 - concat['winner_svm']

In [87]:
concat['winner_nb'] = concat.apply(prob_calc,model='nb', axis=1)
concat['loser_nb'] = 1 - concat['winner_nb']

In [88]:
concat['winner_nn'] = concat.apply(prob_calc,model='nn', axis=1)
concat['loser_nn'] = 1 - concat['winner_nn']

In [89]:
concat

,index,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,score,year,winner_rank,loser_rank,lr,dt,rf,xgb,knn,svm,nb,nn,target,winner_lr,loser_lr,winner_dt,loser_dt,winner_rf,loser_rf,winner_xgb,loser_xgb,winner_knn,loser_knn,winner_svm,loser_svm,winner_nb,loser_nb,winner_nn,loser_nn
0,43661,Adelaide,Hard,Other,20060102.0,Lleyton Hewitt,Jan Hernych,R32,4-6 6-2 6-4,2006.0,74.0,4.0,0.090854,0.270175,0.173877,0.121423,0.222222,0.258194,0.000354,0.090644,0,0.909146,0.090854,0.729825,0.270175,0.826123,0.173877,0.878577,0.121423,0.777778,0.222222,0.741806,0.258194,0.999646,0.000354,0.909356,0.090644
1,43662,Doha,Hard,Other,20060102.0,Roger Federer,Ivo Minar,R32,6-1 6-3,2006.0,1.0,73.0,0.944536,0.780952,0.860848,0.863374,0.793651,0.657106,0.999977,0.934234,1,0.944536,0.055464,0.780952,0.219048,0.860848,0.139152,0.863374,0.136626,0.793651,0.206349,0.657106,0.342894,0.999977,0.000023,0.934234,0.065766
2,43663,Chennai,Hard,Other,20060102.0,Ivan Ljubicic,Guillermo Garcia Lopez,R32,7-6 7-5,2006.0,9.0,91.0,0.754095,0.780952,0.739545,0.725770,0.706349,0.674264,0.985837,0.781877,1,0.754095,0.245905,0.780952,0.219048,0.739545,0.260455,0.725770,0.274230,0.706349,0.293651,0.674264,0.325736,0.985837,0.014163,0.781877,0.218123
3,43664,Adelaide,Hard,Other,20060102.0,Philipp Kohlschreiber,Chris Guccione,R32,6-4 6-3,2006.0,156.0,86.0,0.610163,0.780952,0.560366,0.570551,0.523810,0.486693,0.331592,0.570304,0,0.389837,0.610163,0.219048,0.780952,0.439634,0.560366,0.429449,0.570551,0.476190,0.523810,0.513307,0.486693,0.668408,0.331592,0.429696,0.570304
4,43665,Doha,Hard,Other,20060102.0,Fabrice Santoro,Oliver Marach,R32,6-0 2-6 6-4,2006.0,58.0,127.0,0.692412,0.780952,0.583110,0.672175,0.579365,0.679420,0.927046,0.719571,1,0.692412,0.307588,0.780952,0.219048,0.583110,0.416890,0.672175,0.327825,0.579365,0.420635,0.679420,0.320580,0.927046,0.072954,0.719571,0.280429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45697,89358,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Nardi,RR,2-4 4-3 4-2 1-4 4-2,2023.0,36.0,115.0,0.527965,0.763158,0.766846,0.760311,0.722222,0.648430,0.981146,0.616344,1,0.527965,0.472035,0.763158,0.236842,0.766846,0.233154,0.760311,0.239689,0.722222,0.277778,0.648430,0.351570,0.981146,0.018854,0.616344,0.383656
45698,89359,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Flavio Cobolli,RR,4-1 4-2 4-2,2023.0,36.0,100.0,0.577001,0.700192,0.671077,0.661541,0.642857,0.600003,0.918794,0.669750,1,0.577001,0.422999,0.700192,0.299808,0.671077,0.328923,0.661541,0.338459,0.642857,0.357143,0.600003,0.399997,0.918794,0.081206,0.669750,0.330250
45699,89360,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Dominic Stricker,RR,4-2 3-4 4-2 4-3,2023.0,36.0,94.0,0.520632,0.502863,0.523205,0.588997,0.539683,0.532205,0.866218,0.565114,1,0.520632,0.479368,0.502863,0.497137,0.523205,0.476795,0.588997,0.411003,0.539683,0.460317,0.532205,0.467795,0.866218,0.133782,0.565114,0.434886
45700,89361,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Van Assche,SF,2-4 4-1 4-3 4-3,2023.0,70.0,36.0,0.248567,0.197618,0.274046,0.267798,0.301587,0.232332,0.029877,0.267633,0,0.751433,0.248567,0.802382,0.197618,0.725954,0.274046,0.732202,0.267798,0.698413,0.301587,0.767668,0.232332,0.970123,0.029877,0.732367,0.267633


In [90]:
concat = concat.drop(['lr','dt','rf','xgb','knn','svm','nb','nn','index'], axis=1)

In [91]:
concat

,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,score,year,winner_rank,loser_rank,target,winner_lr,loser_lr,winner_dt,loser_dt,winner_rf,loser_rf,winner_xgb,loser_xgb,winner_knn,loser_knn,winner_svm,loser_svm,winner_nb,loser_nb,winner_nn,loser_nn
0,Adelaide,Hard,Other,20060102.0,Lleyton Hewitt,Jan Hernych,R32,4-6 6-2 6-4,2006.0,74.0,4.0,0,0.909146,0.090854,0.729825,0.270175,0.826123,0.173877,0.878577,0.121423,0.777778,0.222222,0.741806,0.258194,0.999646,0.000354,0.909356,0.090644
1,Doha,Hard,Other,20060102.0,Roger Federer,Ivo Minar,R32,6-1 6-3,2006.0,1.0,73.0,1,0.944536,0.055464,0.780952,0.219048,0.860848,0.139152,0.863374,0.136626,0.793651,0.206349,0.657106,0.342894,0.999977,0.000023,0.934234,0.065766
2,Chennai,Hard,Other,20060102.0,Ivan Ljubicic,Guillermo Garcia Lopez,R32,7-6 7-5,2006.0,9.0,91.0,1,0.754095,0.245905,0.780952,0.219048,0.739545,0.260455,0.725770,0.274230,0.706349,0.293651,0.674264,0.325736,0.985837,0.014163,0.781877,0.218123
3,Adelaide,Hard,Other,20060102.0,Philipp Kohlschreiber,Chris Guccione,R32,6-4 6-3,2006.0,156.0,86.0,0,0.389837,0.610163,0.219048,0.780952,0.439634,0.560366,0.429449,0.570551,0.476190,0.523810,0.513307,0.486693,0.668408,0.331592,0.429696,0.570304
4,Doha,Hard,Other,20060102.0,Fabrice Santoro,Oliver Marach,R32,6-0 2-6 6-4,2006.0,58.0,127.0,1,0.692412,0.307588,0.780952,0.219048,0.583110,0.416890,0.672175,0.327825,0.579365,0.420635,0.679420,0.320580,0.927046,0.072954,0.719571,0.280429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45697,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Nardi,RR,2-4 4-3 4-2 1-4 4-2,2023.0,36.0,115.0,1,0.527965,0.472035,0.763158,0.236842,0.766846,0.233154,0.760311,0.239689,0.722222,0.277778,0.648430,0.351570,0.981146,0.018854,0.616344,0.383656
45698,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Flavio Cobolli,RR,4-1 4-2 4-2,2023.0,36.0,100.0,1,0.577001,0.422999,0.700192,0.299808,0.671077,0.328923,0.661541,0.338459,0.642857,0.357143,0.600003,0.399997,0.918794,0.081206,0.669750,0.330250
45699,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Dominic Stricker,RR,4-2 3-4 4-2 4-3,2023.0,36.0,94.0,1,0.520632,0.479368,0.502863,0.497137,0.523205,0.476795,0.588997,0.411003,0.539683,0.460317,0.532205,0.467795,0.866218,0.133782,0.565114,0.434886
45700,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Van Assche,SF,2-4 4-1 4-3 4-3,2023.0,70.0,36.0,0,0.751433,0.248567,0.802382,0.197618,0.725954,0.274046,0.732202,0.267798,0.698413,0.301587,0.767668,0.232332,0.970123,0.029877,0.732367,0.267633


## Betting data

In [107]:
all_bets =  pd.DataFrame()

for i in range(2016,2024):
    df = pd.read_excel('dane/%s.xlsx'%(i))
    all_bets = pd.concat([all_bets, df], axis=0)

C:\Users\krych\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [108]:
all_bets['year'] = all_bets['Date'].dt.year

In [109]:
all_bets = all_bets.join(all_bets.groupby(['Tournament', 'year'])['Date'].agg(['min']), on=['Tournament','year'])

In [110]:
all_bets

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL,year,min
0,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,6.0,3.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.66,2.10,1.68,2.10,1.62,2.25,1.68,2.31,1.76,2.35,1.66,2.20,2016,2016-01-04
1,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,4.0,6.0,6.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.53,2.37,1.58,2.35,1.53,2.50,1.63,2.40,1.63,2.50,1.57,2.37,2016,2016-01-04
2,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,6.0,2.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.72,2.00,1.82,1.90,1.80,2.00,1.90,1.99,1.90,2.10,1.77,2.00,2016,2016-01-04
3,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Chung H.,Groth S.,51.0,60.0,817.0,782.0,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.83,1.83,1.82,1.90,1.73,2.10,1.93,1.96,1.93,2.10,1.82,1.95,2016,2016-01-04
4,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.28,3.50,1.30,3.40,1.29,3.75,1.31,3.74,1.31,3.80,1.29,3.56,2016,2016-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Alcaraz C.,Medvedev D.,2.0,3.0,8455.0,7200.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.78,2.16,1.78,2.30,1.68,2.19,2023,2023-11-12
2699,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Zverev A.,Rublev A.,8.0,5.0,3585.0,5205.0,6.0,4.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.74,2.23,1.76,2.35,1.66,2.22,2023,2023-11-12
2700,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Sinner J.,Medvedev D.,4.0,3.0,5490.0,7200.0,6.0,3.0,6.0,7.0,6.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.69,2.31,1.71,2.36,1.64,2.26,2023,2023-11-12
2701,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Djokovic N.,Alcaraz C.,1.0,2.0,11445.0,8455.0,6.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.67,2.36,1.70,2.43,1.63,2.28,2023,2023-11-12


In [118]:
all_bets['W1'] = all_bets['W1'].fillna(0)
all_bets['W2'] = all_bets['W2'].fillna(0)
all_bets['W3'] = all_bets['W3'].fillna(0)
all_bets['W4'] = all_bets['W4'].fillna(0)
all_bets['W5'] = all_bets['W5'].fillna(0)
all_bets['L1'] = all_bets['L1'].fillna(0)
all_bets['L2'] = all_bets['L2'].fillna(0)
all_bets['L3'] = all_bets['L3'].fillna(0)
all_bets['L4'] = all_bets['L4'].fillna(0)
all_bets['L5'] = all_bets['L5'].fillna(0)
all_bets['score'] = all_bets.apply(score,axis=1)

In [116]:
def score(row):
    if row['W2'] == 0 and row['L2'] == 0:
        score = '%s-%s'%(str(int(row['W1'])),str(int(row['L1'])))
    elif row['W3'] == 0 and row['L3'] == 0:
        score = '%s-%s %s-%s'%(str(int(row['W1'])),str(int(row['L1'])),str(int(row['W2'])),str(int(row['L2'])))
    elif row['W4'] == 0 and row['L4'] == 0:
        score = '%s-%s %s-%s %s-%s'%(str(int(row['W1'])),str(int(row['L1'])),str(int(row['W2'])),str(int(row['L2'])),str(int(row['W3'])),str(int(row['L3'])))
    elif row['W5'] == 0 and row['L5'] == 0:
        score = '%s-%s %s-%s %s-%s %s-%s'%(str(int(row['W1'])),str(int(row['L1'])),str(int(row['W2'])),str(int(row['L2'])),str(int(row['W3'])),str(int(row['L3'])),str(int(row['W4'])),str(int(row['L4'])))
    else:
        score = '%s-%s %s-%s %s-%s %s-%s %s-%s'%(str(int(row['W1'])),str(int(row['L1'])),str(int(row['W2'])),str(int(row['L2'])),str(int(row['W3'])),str(int(row['L3'])),str(int(row['W4'])),str(int(row['L4'])),str(int(row['W5'])),str(int(row['L5'])))
    return score

In [119]:
all_bets

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL,year,min,score
0,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,6.0,3.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.66,2.10,1.68,2.10,1.62,2.25,1.68,2.31,1.76,2.35,1.66,2.20,2016,2016-01-04,6-3 7-6
1,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,4.0,6.0,6.0,3.0,6.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,Completed,1.53,2.37,1.58,2.35,1.53,2.50,1.63,2.40,1.63,2.50,1.57,2.37,2016,2016-01-04,4-6 6-3 6-2
2,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,6.0,2.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.72,2.00,1.82,1.90,1.80,2.00,1.90,1.99,1.90,2.10,1.77,2.00,2016,2016-01-04,6-2 6-4
3,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Chung H.,Groth S.,51.0,60.0,817.0,782.0,7.0,6.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.83,1.83,1.82,1.90,1.73,2.10,1.93,1.96,1.93,2.10,1.82,1.95,2016,2016-01-04,7-6 6-4
4,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.28,3.50,1.30,3.40,1.29,3.75,1.31,3.74,1.31,3.80,1.29,3.56,2016,2016-01-04,6-4 6-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Alcaraz C.,Medvedev D.,2.0,3.0,8455.0,7200.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.78,2.16,1.78,2.30,1.68,2.19,2023,2023-11-12,6-4 6-4
2699,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Zverev A.,Rublev A.,8.0,5.0,3585.0,5205.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.74,2.23,1.76,2.35,1.66,2.22,2023,2023-11-12,6-4 6-4
2700,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Sinner J.,Medvedev D.,4.0,3.0,5490.0,7200.0,6.0,3.0,6.0,7.0,6.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.69,2.31,1.71,2.36,1.64,2.26,2023,2023-11-12,6-3 6-7 6-1
2701,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Djokovic N.,Alcaraz C.,1.0,2.0,11445.0,8455.0,6.0,3.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.67,2.36,1.70,2.43,1.63,2.28,2023,2023-11-12,6-3 6-2


In [120]:
pd.set_option('display.max_rows', 10)
all_bets[['Winner','Loser','year','Surface','WRank','LRank','score']].value_counts().reset_index(name='count')

,Winner,Loser,year,Surface,WRank,LRank,score,count
0,Agamenone F.,Baez S.,2022,Clay,136.0,32.0,3-6 6-1 7-5,1
1,Nishikori K.,Norrie C.,2021,Hard,67.0,29.0,3-6 6-3 6-3,1
2,Nishikori K.,Paire B.,2019,Hard,6.0,65.0,6-4 6-3,1
3,Nishikori K.,Paire B.,2019,Clay,7.0,38.0,6-2 6-7 6-2 6-7 7-5,1
4,Nishikori K.,Paire B.,2018,Hard,12.0,63.0,6-3 7-5,1
...,...,...,...,...,...,...,...,...
19537,Garin C.,Galan D.E.,2021,Clay,22.0,121.0,6-4 6-3,1
19538,Garin C.,Fritz T.,2022,Clay,29.0,13.0,6-2 6-7 6-3,1
19539,Garin C.,Eubanks C.,2019,Hard,32.0,188.0,3-6 7-6 6-4 6-7 6-3,1
19540,Garin C.,Escobedo E.,2021,Hard,17.0,178.0,7-5 6-2,1


In [121]:
all_bets['winner_name'] = all_bets['Winner'].str.rsplit(' ',1).str[0]
all_bets['loser_name'] = all_bets['Loser'].str.rsplit(' ',1).str[0]

C:\Users\krych\AppData\Local\Temp\ipykernel_11400\3631672389.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  all_bets['winner_name'] = all_bets['Winner'].str.rsplit(' ',1).str[0]
C:\Users\krych\AppData\Local\Temp\ipykernel_11400\3631672389.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  all_bets['loser_name'] = all_bets['Loser'].str.rsplit(' ',1).str[0]


In [122]:
all_bets

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL,year,min,score,winner_name,loser_name
0,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Dimitrov G.,Simon G.,28.0,15.0,1360.0,2145.0,6.0,3.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.66,2.10,1.68,2.10,1.62,2.25,1.68,2.31,1.76,2.35,1.66,2.20,2016,2016-01-04,6-3 7-6,Dimitrov,Simon
1,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kudla D.,Smith J.P.,69.0,129.0,719.0,459.0,4.0,6.0,6.0,3.0,6.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,Completed,1.53,2.37,1.58,2.35,1.53,2.50,1.63,2.40,1.63,2.50,1.57,2.37,2016,2016-01-04,4-6 6-3 6-2,Kudla,Smith
2,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kamke T.,Mitchell B.,277.0,231.0,185.0,225.0,6.0,2.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.72,2.00,1.82,1.90,1.80,2.00,1.90,1.99,1.90,2.10,1.77,2.00,2016,2016-01-04,6-2 6-4,Kamke,Mitchell
3,1,Brisbane,Brisbane International,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Chung H.,Groth S.,51.0,60.0,817.0,782.0,7.0,6.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.83,1.83,1.82,1.90,1.73,2.10,1.93,1.96,1.93,2.10,1.82,1.95,2016,2016-01-04,7-6 6-4,Chung,Groth
4,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Goffin D.,Bellucci T.,16.0,37.0,1880.0,1105.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.28,3.50,1.30,3.40,1.29,3.75,1.31,3.74,1.31,3.80,1.29,3.56,2016,2016-01-04,6-4 6-4,Goffin,Bellucci
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Alcaraz C.,Medvedev D.,2.0,3.0,8455.0,7200.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.78,2.16,1.78,2.30,1.68,2.19,2023,2023-11-12,6-4 6-4,Alcaraz,Medvedev
2699,64,Turin,Masters Cup,2023-11-17,Masters Cup,Indoor,Hard,Round Robin,3.0,Zverev A.,Rublev A.,8.0,5.0,3585.0,5205.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.74,2.23,1.76,2.35,1.66,2.22,2023,2023-11-12,6-4 6-4,Zverev,Rublev
2700,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Sinner J.,Medvedev D.,4.0,3.0,5490.0,7200.0,6.0,3.0,6.0,7.0,6.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,Completed,1.67,2.20,NaN,NaN,NaN,NaN,1.69,2.31,1.71,2.36,1.64,2.26,2023,2023-11-12,6-3 6-7 6-1,Sinner,Medvedev
2701,64,Turin,Masters Cup,2023-11-18,Masters Cup,Indoor,Hard,Semifinals,3.0,Djokovic N.,Alcaraz C.,1.0,2.0,11445.0,8455.0,6.0,3.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.62,2.30,NaN,NaN,NaN,NaN,1.67,2.36,1.70,2.43,1.63,2.28,2023,2023-11-12,6-3 6-2,Djokovic,Alcaraz


In [123]:
concat2 = concat[concat['year']>2015]

In [124]:
concat2['winner_name2'] = concat2['winner_name'].str.split(' ',1).str[1]
concat2['loser_name2'] = concat2['loser_name'].str.split(' ',1).str[1]

C:\Users\krych\AppData\Local\Temp\ipykernel_11400\537061259.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  concat2['winner_name2'] = concat2['winner_name'].str.split(' ',1).str[1]
C:\Users\krych\AppData\Local\Temp\ipykernel_11400\537061259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat2['winner_name2'] = concat2['winner_name'].str.split(' ',1).str[1]
C:\Users\krych\AppData\Local\Temp\ipykernel_11400\537061259.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  concat2['loser_name2'] = concat2['loser_name'].str.split(' ',1).str[1]
C:\Users\krych\AppData\

In [125]:
concat2

,tourney_name,surface,tourney_level,tourney_date,winner_name,loser_name,round,score,year,winner_rank,loser_rank,target,winner_lr,loser_lr,winner_dt,loser_dt,winner_rf,loser_rf,winner_xgb,loser_xgb,winner_knn,loser_knn,winner_svm,loser_svm,winner_nb,loser_nb,winner_nn,loser_nn,winner_name2,loser_name2
26285,Doha,Hard,Other,20160104.0,Rafael Nadal,Pablo Carreno Busta,R32,6-7 6-3 6-1,2016.0,5.0,67.0,1,0.952721,0.047279,0.841724,0.158276,0.892324,0.107676,0.927964,0.072036,0.912698,0.087302,0.761370,0.238630,0.999986,0.000014,0.928160,0.071840,Nadal,Carreno Busta
26286,Brisbane,Hard,Other,20160104.0,Mikhail Kukushkin,Denis Istomin,R32,6-2 7-5,2016.0,65.0,61.0,1,0.437077,0.562923,0.444165,0.555835,0.362359,0.637641,0.386247,0.613753,0.380952,0.619048,0.327385,0.672615,0.158810,0.841190,0.408565,0.591435,Kukushkin,Istomin
26287,Chennai,Hard,Other,20160104.0,Ramkumar Ramanathan,Daniel Gimeno Traver,R32,6-2 6-0,2016.0,248.0,98.0,1,0.512154,0.487846,0.317356,0.682644,0.296012,0.703988,0.321097,0.678903,0.388889,0.611111,0.317678,0.682322,0.000832,0.999168,0.392593,0.607407,Ramanathan,Gimeno Traver
26288,Doha,Hard,Other,20160104.0,Robin Haase,Aslan Karatsev,R32,5-7 6-1 7-5,2016.0,195.0,66.0,0,0.635233,0.364767,0.682644,0.317356,0.742628,0.257372,0.726341,0.273659,0.658730,0.341270,0.767187,0.232813,0.980017,0.019983,0.700652,0.299348,Haase,Karatsev
26289,Brisbane,Hard,Other,20160104.0,Radek Stepanek,Dusan Lajovic,R32,6-0 6-3,2016.0,197.0,76.0,1,0.499156,0.500844,0.643597,0.356403,0.575835,0.424165,0.520601,0.479399,0.539683,0.460317,0.551982,0.448018,0.873468,0.126532,0.579895,0.420105,Stepanek,Lajovic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45697,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Nardi,RR,2-4 4-3 4-2 1-4 4-2,2023.0,36.0,115.0,1,0.527965,0.472035,0.763158,0.236842,0.766846,0.233154,0.760311,0.239689,0.722222,0.277778,0.648430,0.351570,0.981146,0.018854,0.616344,0.383656,Fils,Nardi
45698,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Flavio Cobolli,RR,4-1 4-2 4-2,2023.0,36.0,100.0,1,0.577001,0.422999,0.700192,0.299808,0.671077,0.328923,0.661541,0.338459,0.642857,0.357143,0.600003,0.399997,0.918794,0.081206,0.669750,0.330250,Fils,Cobolli
45699,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Dominic Stricker,RR,4-2 3-4 4-2 4-3,2023.0,36.0,94.0,1,0.520632,0.479368,0.502863,0.497137,0.523205,0.476795,0.588997,0.411003,0.539683,0.460317,0.532205,0.467795,0.866218,0.133782,0.565114,0.434886,Fils,Stricker
45700,NextGen Finals,Hard,Season-ending events,20231127.0,Arthur Fils,Luca Van Assche,SF,2-4 4-1 4-3 4-3,2023.0,70.0,36.0,0,0.751433,0.248567,0.802382,0.197618,0.725954,0.274046,0.732202,0.267798,0.698413,0.301587,0.767668,0.232332,0.970123,0.029877,0.732367,0.267633,Fils,Van Assche


In [126]:
concat2[['winner_name2','loser_name2','year','surface','score','winner_rank','loser_rank']].value_counts().reset_index(name='count')

,winner_name2,loser_name2,year,surface,score,winner_rank,loser_rank,count
0,Agamenone,Baez,2022.0,Clay,3-6 6-1 7-5,136.0,32.0,1
1,Nakashima,Lehecka,2022.0,Hard,4-3 4-3 4-2,49.0,74.0,1
2,Nakashima,Laaksonen,2021.0,Hard,7-6 6-7 6-3,99.0,79.0,1
3,Nakashima,Kuhn,2022.0,Grass,6-3 6-7 6-3 6-2,261.0,56.0,1
4,Nakashima,Kudla,2022.0,Hard,6-3 6-4,104.0,69.0,1
...,...,...,...,...,...,...,...,...
19412,Gasquet,Lestienne,2018.0,Hard,6-4 6-2,167.0,25.0,1
19413,Gasquet,Lennard Struff,2018.0,Hard,6-4 6-7 7-6,58.0,28.0,1
19414,Gasquet,Lennard Struff,2016.0,Hard,6-2 6-4,98.0,19.0,1
19415,Gasquet,Landaluce,2023.0,Clay,6-2 6-1,43.0,894.0,1


In [300]:
concat2['tourney_date'] = pd.to_datetime(concat2['tourney_date'], format='%Y%m%d')

C:\Users\krych\AppData\Local\Temp\ipykernel_6808\69454723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat2['tourney_date'] = pd.to_datetime(concat2['tourney_date'], format='%Y%m%d')


In [144]:
for index, row in concat2.iterrows():
    if row['target']==0:
        concat2['winner_rank'][index], concat2['loser_rank'][index] = concat2['loser_rank'][index],concat2['winner_rank'][index]

C:\Users\krych\AppData\Local\Temp\ipykernel_11400\2180555410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat2['winner_rank'][index], concat2['loser_rank'][index] = concat2['loser_rank'][index],concat2['winner_rank'][index]


In [129]:
concat2 = concat2.replace('Manuel Cerundolo', 'Cerundolo')
all_bets = all_bets.replace('Auger-Aliassime', 'Auger Aliassime')
all_bets =all_bets.replace('Struff', 'Lennard Struff')
all_bets =all_bets.replace('Wolf', 'J Wolf')
all_bets =all_bets.replace('Kwon', 'Woo Kwon')
all_bets =all_bets.replace('Etcheverry', 'Martin Etcheverry')
all_bets =all_bets.replace('Galan', 'Elahi Galan')
all_bets =all_bets.replace('O Connell', 'Oconnell')
all_bets =all_bets.replace('Varillas', 'Pablo Varillas')
all_bets =all_bets.replace('Huesler', 'Andrea Huesler')
all_bets =all_bets.replace('Zayid', 'Shannan Zayid')
all_bets =all_bets.replace('Herbert', 'Hugues Herbert')
all_bets =all_bets.replace('Tseng C.', 'Hsin Tseng')
all_bets =all_bets.replace('Tirante', 'Agustin Tirante')
all_bets =all_bets.replace('Olivieri', 'Alberto Olivieri')
all_bets =all_bets.replace('Ramos-Vinolas', 'Ramos')
all_bets =all_bets.replace('Varillas J.', 'Pablo Varillas')
all_bets =all_bets.replace('Tseng', 'Hsin Tseng')
all_bets =all_bets.replace('Hsu', 'Hsiou Hsu')
all_bets =all_bets.replace('Rehberg', 'Hans Rehberg')
all_bets =all_bets.replace('Madaras', 'Nicolae Madaras')
all_bets =all_bets.replace('Mukund', 'Kumar Mukund')
all_bets =all_bets.replace('Kwiatkowski', 'Son Kwiatkowski')
all_bets =all_bets.replace('Mo', 'Cong Mo')
all_bets =all_bets.replace('Nam', 'Sung Nam')
all_bets =all_bets.replace('Stebe', 'Marcel Stebe')
all_bets =all_bets.replace('Lu', 'Hsun Lu')
all_bets =all_bets.replace('Smith', 'Patrick Smith')
all_bets =all_bets.replace('Londero', 'Ignacio Londero')
concat2 =concat2.replace('Lin Wu', 'Wu')
all_bets =all_bets.replace('Barrios', 'Barrios Vera')
all_bets =all_bets.replace('Ficovich', 'Pablo Ficovich')
all_bets =all_bets.replace('Moroni', 'Marco Moroni')
all_bets =all_bets.replace('Del Potro', 'Martin del Potro')
all_bets =all_bets.replace('Meligeni Rodrigues', 'Meligeni Alves')
all_bets =all_bets.replace('Hernandez', 'Alejandro Hernandez Serrano')
all_bets =all_bets.replace('Pucinelli de Almeida', 'Pucinelli De Almeida')
all_bets =all_bets.replace('Rodriguez Taverna', 'Fa Rodriguez Taverna')
all_bets =all_bets.replace('Hong', 'Chan Hong')
all_bets =all_bets.replace('Mpetshi', 'Mpetshi Perricard')
all_bets =all_bets.replace('Bu', 'Yunchaokete')

In [147]:
merged = pd.merge(concat2, all_bets,  how='inner', left_on=['winner_name2','loser_name2','year','score','surface','winner_rank','loser_rank'], right_on = ['winner_name','loser_name','year','score','Surface','WRank','LRank'])

In [150]:
merged

,tourney_name,surface,tourney_level,tourney_date,winner_name_x,loser_name_x,round,score,year,winner_rank,loser_rank,target,winner_lr,loser_lr,winner_dt,loser_dt,winner_rf,loser_rf,winner_xgb,loser_xgb,winner_knn,loser_knn,winner_svm,loser_svm,winner_nb,loser_nb,winner_nn,loser_nn,winner_name2,loser_name2,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,MaxW,MaxL,AvgW,AvgL,min,winner_name_y,loser_name_y
0,Brisbane,Hard,Other,20160104.0,Mikhail Kukushkin,Denis Istomin,R32,6-2 7-5,2016.0,65.0,61.0,1,0.437077,0.562923,0.444165,0.555835,0.362359,0.637641,0.386247,0.613753,0.380952,0.619048,0.327385,0.672615,0.158810,0.841190,0.408565,0.591435,Kukushkin,Istomin,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Kukushkin M.,Istomin D.,65.0,61.0,762.0,781.0,6.0,2.0,7.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,2.00,1.72,2.05,1.72,2.10,1.73,2.14,1.78,2.18,1.79,2.09,1.71,2016-01-04,Kukushkin,Istomin
1,Doha,Hard,Other,20160104.0,Robin Haase,Aslan Karatsev,R32,5-7 6-1 7-5,2016.0,66.0,195.0,0,0.635233,0.364767,0.682644,0.317356,0.742628,0.257372,0.726341,0.273659,0.658730,0.341270,0.767187,0.232813,0.980017,0.019983,0.700652,0.299348,Haase,Karatsev,3,Doha,Qatar Exxon Mobil Open,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Haase R.,Karatsev A.,66.0,195.0,740.0,254.0,5.0,7.0,6.0,1.0,7.0,5.0,0.0,0.0,0.0,0.0,2.0,1.0,Completed,1.50,2.50,1.52,2.45,1.53,2.50,1.50,2.76,1.58,2.76,1.52,2.49,2016-01-04,Haase,Karatsev
2,Brisbane,Hard,Other,20160104.0,Radek Stepanek,Dusan Lajovic,R32,6-0 6-3,2016.0,197.0,76.0,1,0.499156,0.500844,0.643597,0.356403,0.575835,0.424165,0.520601,0.479399,0.539683,0.460317,0.551982,0.448018,0.873468,0.126532,0.579895,0.420105,Stepanek,Lajovic,1,Brisbane,Brisbane International,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Stepanek R.,Lajovic D.,197.0,76.0,252.0,678.0,6.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,2.00,1.72,1.90,1.80,2.00,1.80,2.00,1.89,2.05,1.90,1.94,1.82,2016-01-04,Stepanek,Lajovic
3,Chennai,Hard,Other,20160104.0,Luca Vanni,Jan Lennard Struff,R32,6-4 7-6,2016.0,106.0,107.0,0,0.493474,0.506526,0.555835,0.444165,0.450242,0.549758,0.390731,0.609269,0.563492,0.436508,0.598594,0.401406,0.740654,0.259346,0.406331,0.593669,Vanni,Lennard Struff,2,Chennai,Chennai Open,2016-01-05,ATP250,Outdoor,Hard,1st Round,3.0,Vanni L.,Struff J.L.,106.0,107.0,552.0,547.0,6.0,4.0,7.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,2.20,1.61,2.10,1.70,2.00,1.80,2.10,1.81,2.20,1.81,2.09,1.71,2016-01-04,Vanni,Lennard Struff
4,Doha,Hard,Other,20160104.0,Andrey Kuznetsov,Marsel Ilhan,R32,6-2 6-4,2016.0,79.0,114.0,0,0.600416,0.399584,0.682644,0.317356,0.627727,0.372273,0.667556,0.332444,0.603175,0.396825,0.709722,0.290278,0.727154,0.272846,0.612410,0.387590,Kuznetsov,Ilhan,3,Doha,Qatar Exxon Mobil Open,2016-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Kuznetsov An.,Ilhan M.,79.0,114.0,663.0,520.0,6.0,2.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.50,2.50,1.52,2.45,1.53,2.38,1.59,2.49,1.62,2.62,1.54,2.43,2016-01-04,Kuznetsov,Ilhan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18151,Tour Finals,Hard,Other,20231113.0,Jannik Sinner,Stefanos Tsitsipas,RR,6-4 6-4,2023.0,4.0,6.0,1,0.649353,0.350647,0.700192,0.299808,0.687895,0.312105,0.716252,0.283748,0.619048,0.380952,0.727980,0.272020,0.912216,0.087784,0.676799,0.323201,Sinner,Tsitsipas,64,Turin,Masters Cup,2023-11-12,Masters Cup,Indoor,Hard,Round Robin,3.0,Sinner J.,Tsitsipas S.,4.0,6.0,5490.0,4435.0,6.0,4.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Completed,1.36,3.20,NaN,NaN,NaN,NaN,1.39,3.26,1.44,3.33,1.36,3.14,2023-11-12,Sinner,Tsitsipas
18152,Tour Finals,Hard,Other,20

In [149]:
merged.to_csv('Matches_with_bets2.csv',index=False)